In [1]:
# ----
library(tidyverse)
library(lubridate)

setwd("/home/tumtse/Documents/haowu/DRZ/")
runID="output-berlin-v5.5-1pct-baseCase_200_simple"
#runID="output-berlin-v5.5-1pct-policyCase1_200_simple"
#runID="output-berlin-v5.5-1pct-policyCase3_200_simple"




# ----
runDirectory = paste("events_Biao", runID, "berlin-v5.5-1pct.output_trips.csv.gz", sep = "/")
#runDirectory = paste"events_Biao", runID, "berlin-v5.5-1pct.output_trips-berlin.csv.gz", sep = "/")


trips <-read_delim(runDirectory,
                            delim=";",
                            # sep = "\t",
                            locale=locale(decimal_mark = "."),
                            col_types = cols(
                              #x = col_double(),
                              #y = col_double()
                              person = col_character(),
                              #vehicleId = col_character(),
                              #fromLinkId = col_double(),
                              #start_x = col_double(),
                              #start_y = col_double()
                              #toLinkId = col_double(),
                              #toX = col_double(),
                              #toY = col_double(),
                              #waitTime = col_double(),
                              #arrivalTime = col_double(),
                              #travelTime = col_double(),
                              #traveled_distance = col_double(),
                              #direcTravelDistance_m = col_double()
                            ))

#view(trips)
#trips


residents <-read_delim("matsim-berlin-CarFreeZone/scenarios/berlin-v5.5-1pct/input/carFreeZone/PlanA/IDLists/berlinAgents/personInternalIDsList.txt", 
                          delim="\t",
                          # sep = "\t",
                          locale=locale(decimal_mark = "."),
                          col_names = c("person"),
                          col_types = cols(person=col_character())
                          )
#view(residents)
#residents

workers <-read_delim("matsim-berlin-CarFreeZone/scenarios/berlin-v5.5-1pct/input/carFreeZone/PlanA/IDLists/berlinAgents/workerIDsList.txt", 
                       delim="\t",
                       # sep = "\t",
                       locale=locale(decimal_mark = "."),
                       col_names = c("person"),
                       col_types = cols(person=col_character())
                       )
#view(workers)
#workers

visitors <-read_delim("matsim-berlin-CarFreeZone/scenarios/berlin-v5.5-1pct/input/carFreeZone/PlanA/IDLists/berlinAgents/agentsDoingOtherActivitiesIDsList.txt", 
                     delim="\t",
                     # sep = "\t",
                     locale=locale(decimal_mark = "."),
                     col_names = c("person"),
                     col_types = cols(person=col_character())
                     )
#view(visitors)
#visitors

passers <-read_delim("matsim-berlin-CarFreeZone/scenarios/berlin-v5.5-1pct/input/carFreeZone/PlanA/IDLists/berlinAgents/agentsWithoutActivitiesIDsList.txt", 
                      delim="\t",
                      # sep = "\t",
                      locale=locale(decimal_mark = "."),
                      col_names = c("person"),
                      col_types = cols(person=col_character())
                      )
#view(passers)
#passers

nonAffectedAgents <-read_delim("matsim-berlin-CarFreeZone/scenarios/berlin-v5.5-1pct/input/carFreeZone/PlanA/IDLists/berlinAgents/nonAffectedAgentsIDsList.txt", 
                     delim="\t",
                     # sep = "\t",
                     locale=locale(decimal_mark = "."),
                     col_names = c("person"),
                     col_types = cols(person=col_character())
                     )
#view(nonAffectedAgents)
#nonAffectedAgents


# merging id of captured agents from residents, workers, visitors, passers, nonAffectedAgents

# First, make sure you have dplyr installed and loaded
if (!require("dplyr")) install.packages("dplyr")
library(dplyr)
# Merge the data frames
capturedAgents <- bind_rows(residents, workers, visitors, passers, nonAffectedAgents)
capturedAgents <- capturedAgents %>%
  distinct(person)

# print the captured agents to a txt file
outputDirectory = paste("matsim-berlin-CarFreeZone/scenarios/berlin-v5.5-1pct/input/carFreeZone/PlanA/IDLists/berlinAgents/capturedAgentsIDsList.txt", sep = "/")
write.table(capturedAgents, outputDirectory, row.names = FALSE, col.names = FALSE, quote = FALSE)

# View the merged data
#view(capturedAgents)
#capturedAgents




allAgents <-read_delim("matsim-berlin-CarFreeZone/scenarios/berlin-v5.5-1pct/input/carFreeZone/PlanA/IDLists/berlinAgents/allAgentIDsList.txt", 
                     delim="\t",
                     # sep = "\t",
                     locale=locale(decimal_mark = "."),
                     col_names = c("person"),
                     col_types = cols(person=col_character())
                     )
#view(allAgents)
#allAgents


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
# ----
# residents
joined_trips_residents <- semi_join(trips, residents)
#view(joined_trips_residents)


joined_trips_residents_1 <- joined_trips_residents %>% 
  mutate(travel_time = trav_time+wait_time) %>% 
  group_by(person) %>%
  mutate(mean_traveled_distance_person=mean(traveled_distance)) %>% 
  mutate(mean_travel_time_person=mean(travel_time))

joined_trips_residents_1 <- joined_trips_residents_1 %>% 
  select(person, mean_traveled_distance_person, mean_travel_time_person)
joined_trips_residents_1 <- distinct(joined_trips_residents_1)


joined_trips_residents_travel_time <- joined_trips_residents_1 %>%
  group_by() %>%
  summarise(mean_travel_time_person_mean = mean(mean_travel_time_person, na.rm = TRUE))

joined_trips_residents_travel_distance <- joined_trips_residents_1 %>%
  group_by() %>%
  summarise(mean_traveled_distance_person_mean = mean(mean_traveled_distance_person, na.rm = TRUE))


#view(joined_trips_residents_1)
#view(joined_trips_residents_travel_time)
#view(joined_trips_residents_travel_distance)
#write.csv(joined_trips_residents_1,"/Users/haowu/Downloads/residents.csv", row.names = FALSE)


# ----
# workers
joined_trips_workers <- semi_join(trips, workers)
#view(joined_trips_workers)


joined_trips_workers_1 <- joined_trips_workers %>% 
  mutate(travel_time = trav_time+wait_time) %>% 
  group_by(person) %>%
  mutate(mean_traveled_distance_person=mean(traveled_distance)) %>% 
  mutate(mean_travel_time_person=mean(travel_time))

joined_trips_workers_1 <- joined_trips_workers_1 %>% 
  select(person, mean_traveled_distance_person, mean_travel_time_person)
joined_trips_workers_1 <- distinct(joined_trips_workers_1)


joined_trips_workers_travel_time <- joined_trips_workers_1 %>%
  group_by() %>%
  summarise(mean_travel_time_person_mean = mean(mean_travel_time_person, na.rm = TRUE))

joined_trips_workers_travel_distance <- joined_trips_workers_1 %>%
  group_by() %>%
  summarise(mean_traveled_distance_person_mean = mean(mean_traveled_distance_person, na.rm = TRUE))


#view(joined_trips_workers_1)
#view(joined_trips_workers_travel_time)
#view(joined_trips_workers_travel_distance)


# ----
# visitors
joined_trips_visitors <- semi_join(trips, visitors)
#view(joined_trips_visitors)


joined_trips_visitors_1 <- joined_trips_visitors %>% 
  mutate(travel_time = trav_time+wait_time) %>% 
  group_by(person) %>%
  mutate(mean_traveled_distance_person=mean(traveled_distance)) %>% 
  mutate(mean_travel_time_person=mean(travel_time))

joined_trips_visitors_1 <- joined_trips_visitors_1 %>% 
  select(person, mean_traveled_distance_person, mean_travel_time_person)
joined_trips_visitors_1 <- distinct(joined_trips_visitors_1)


joined_trips_visitors_travel_time <- joined_trips_visitors_1 %>%
  group_by() %>%
  summarise(mean_travel_time_person_mean = mean(mean_travel_time_person, na.rm = TRUE))

joined_trips_visitors_travel_distance <- joined_trips_visitors_1 %>%
  group_by() %>%
  summarise(mean_traveled_distance_person_mean = mean(mean_traveled_distance_person, na.rm = TRUE))


#view(joined_trips_visitors_1)
#view(joined_trips_visitors_travel_time)
#view(joined_trips_visitors_travel_distance)


# ----
# passers
joined_trips_passers <- semi_join(trips, passers)
#view(joined_trips_passers)


joined_trips_passers_1 <- joined_trips_passers %>% 
  mutate(travel_time = trav_time+wait_time) %>% 
  group_by(person) %>%
  mutate(mean_traveled_distance_person=mean(traveled_distance)) %>% 
  mutate(mean_travel_time_person=mean(travel_time))

joined_trips_passers_1 <- joined_trips_passers_1 %>% 
  select(person, mean_traveled_distance_person, mean_travel_time_person)
joined_trips_passers_1 <- distinct(joined_trips_passers_1)


joined_trips_passers_travel_time <- joined_trips_passers_1 %>%
  group_by() %>%
  summarise(mean_travel_time_person_mean = mean(mean_travel_time_person, na.rm = TRUE))

joined_trips_passers_travel_distance <- joined_trips_passers_1 %>%
  group_by() %>%
  summarise(mean_traveled_distance_person_mean = mean(mean_traveled_distance_person, na.rm = TRUE))


#view(joined_trips_passers_1)
#view(joined_trips_passers_travel_time)
#view(joined_trips_passers_travel_distance)


# ----
# nonAffectedAgents
joined_trips_nonAffectedAgents <- semi_join(trips, nonAffectedAgents)
#view(joined_trips_nonAffectedAgents)


joined_trips_nonAffectedAgents_1 <- joined_trips_nonAffectedAgents %>% 
  mutate(travel_time = trav_time+wait_time) %>% 
  group_by(person) %>%
  mutate(mean_traveled_distance_person=mean(traveled_distance)) %>% 
  mutate(mean_travel_time_person=mean(travel_time))

joined_trips_nonAffectedAgents_1 <- joined_trips_nonAffectedAgents_1 %>% 
  select(person, mean_traveled_distance_person, mean_travel_time_person)
joined_trips_nonAffectedAgents_1 <- distinct(joined_trips_nonAffectedAgents_1)


joined_trips_nonAffectedAgents_travel_time <- joined_trips_nonAffectedAgents_1 %>%
  group_by() %>%
  summarise(mean_travel_time_person_mean = mean(mean_travel_time_person, na.rm = TRUE))

joined_trips_nonAffectedAgents_travel_distance <- joined_trips_nonAffectedAgents_1 %>%
  group_by() %>%
  summarise(mean_traveled_distance_person_mean = mean(mean_traveled_distance_person, na.rm = TRUE))


#view(joined_trips_nonAffectedAgents_1)
#view(joined_trips_nonAffectedAgents_travel_time)
#view(joined_trips_nonAffectedAgents_travel_distance)


# ----
# capturedAgents
joined_trips_capturedAgents <- semi_join(trips, capturedAgents)
#view(joined_trips_capturedAgents)


joined_trips_capturedAgents_1 <- joined_trips_capturedAgents %>% 
  mutate(travel_time = trav_time+wait_time) %>% 
  group_by(person) %>%
  mutate(mean_traveled_distance_person=mean(traveled_distance)) %>% 
  mutate(mean_travel_time_person=mean(travel_time))

joined_trips_capturedAgents_1 <- joined_trips_capturedAgents_1 %>% 
  select(person, mean_traveled_distance_person, mean_travel_time_person)
joined_trips_capturedAgents_1 <- distinct(joined_trips_capturedAgents_1)


joined_trips_capturedAgents_travel_time <- joined_trips_capturedAgents_1 %>%
  group_by() %>%
  summarise(mean_travel_time_person_mean = mean(mean_travel_time_person, na.rm = TRUE))

joined_trips_capturedAgents_travel_distance <- joined_trips_capturedAgents_1 %>%
  group_by() %>%
  summarise(mean_traveled_distance_person_mean = mean(mean_traveled_distance_person, na.rm = TRUE))


#view(joined_trips_capturedAgents_1)
#view(joined_trips_capturedAgents_travel_time)
#view(joined_trips_capturedAgents_travel_distance)


# ----
# allAgents
joined_trips_allAgents <- semi_join(trips, allAgents)
#view(joined_trips_allAgents)


joined_trips_allAgents_1 <- joined_trips_allAgents %>% 
  mutate(travel_time = trav_time+wait_time) %>% 
  group_by(person) %>%
  mutate(mean_traveled_distance_person=mean(traveled_distance)) %>% 
  mutate(mean_travel_time_person=mean(travel_time))

joined_trips_allAgents_1 <- joined_trips_allAgents_1 %>% 
  select(person, mean_traveled_distance_person, mean_travel_time_person)
joined_trips_allAgents_1 <- distinct(joined_trips_allAgents_1)


joined_trips_allAgents_travel_time <- joined_trips_allAgents_1 %>%
  group_by() %>%
  summarise(mean_travel_time_person_mean = mean(mean_travel_time_person, na.rm = TRUE))

joined_trips_allAgents_travel_distance <- joined_trips_allAgents_1 %>%
  group_by() %>%
  summarise(mean_traveled_distance_person_mean = mean(mean_traveled_distance_person, na.rm = TRUE))


#view(joined_trips_allAgents_1)
#view(joined_trips_allAgents_travel_time)
#view(joined_trips_allAgents_travel_distance)




# ----
joined_trips_residents_travel_time_value <- as.numeric(unlist(joined_trips_residents_travel_time[1,1]))
joined_trips_residents_travel_distance_value <- as.numeric(unlist(joined_trips_residents_travel_distance[1,1]))

joined_trips_workers_travel_time_value <- as.numeric(unlist(joined_trips_workers_travel_time[1,1]))
joined_trips_workers_travel_distance_value <- as.numeric(unlist(joined_trips_workers_travel_distance[1,1]))

joined_trips_visitors_travel_time_value <- as.numeric(unlist(joined_trips_visitors_travel_time[1,1]))
joined_trips_visitors_travel_distance_value <- as.numeric(unlist(joined_trips_visitors_travel_distance[1,1]))

joined_trips_passers_travel_time_value <- as.numeric(unlist(joined_trips_passers_travel_time[1,1]))
joined_trips_passers_travel_distance_value <- as.numeric(unlist(joined_trips_passers_travel_distance[1,1]))

joined_trips_nonAffectedAgents_travel_time_value <- as.numeric(unlist(joined_trips_nonAffectedAgents_travel_time[1,1]))
joined_trips_nonAffectedAgents_travel_distance_value <- as.numeric(unlist(joined_trips_nonAffectedAgents_travel_distance[1,1]))

joined_trips_capturedAgents_travel_time_value <- as.numeric(unlist(joined_trips_capturedAgents_travel_time[1,1]))
joined_trips_capturedAgents_travel_distance_value <- as.numeric(unlist(joined_trips_capturedAgents_travel_distance[1,1]))

joined_trips_allAgents_travel_time_value <- as.numeric(unlist(joined_trips_allAgents_travel_time[1,1]))
joined_trips_allAgents_travel_distance_value <- as.numeric(unlist(joined_trips_allAgents_travel_distance[1,1]))

# ----
joined_trips_residents <- joined_trips_residents  %>%
  select(person)  %>%
  distinct()
number_residents <- count(joined_trips_residents, "person")
number_residents <- as.numeric(unlist(number_residents[1,2]))

joined_trips_workers <- joined_trips_workers  %>%
  select(person)  %>%
  distinct()
number_workers <- count(joined_trips_workers, "person")
number_workers <- as.numeric(unlist(number_workers[1,2]))

joined_trips_visitors <- joined_trips_visitors  %>%
  select(person)  %>%
  distinct()
number_visitors <- count(joined_trips_visitors, "person")
number_visitors <- as.numeric(unlist(number_visitors[1,2]))

joined_trips_passers <- joined_trips_passers  %>%
  select(person)  %>%
  distinct()
number_passers <- count(joined_trips_passers, "person")
number_passers <- as.numeric(unlist(number_passers[1,2]))

joined_trips_nonAffectedAgents <- joined_trips_nonAffectedAgents  %>%
  select(person)  %>%
  distinct()
number_nonAffectedAgents <- count(joined_trips_nonAffectedAgents, "person")
number_nonAffectedAgents <- as.numeric(unlist(number_nonAffectedAgents[1,2]))

joined_trips_capturedAgents <- joined_trips_capturedAgents  %>%
  select(person)  %>%
  distinct()
number_capturedAgents <- count(joined_trips_capturedAgents, "person")
number_capturedAgents <- as.numeric(unlist(number_capturedAgents[1,2]))

joined_trips_allAgents <- joined_trips_allAgents  %>%
  select(person)  %>%
  distinct()
number_allAgents <- count(joined_trips_allAgents, "person")
number_allAgents <- as.numeric(unlist(number_allAgents[1,2]))

# ----
agent_group <- c('residents','workers','visitors','passers','nonAffectedAgents','capturedAgents','allAgents')
number_agent_group <- c(number_residents, number_workers, number_visitors, number_passers, number_nonAffectedAgents, number_capturedAgents, number_allAgents)
travel_time <- c(joined_trips_residents_travel_time_value, joined_trips_workers_travel_time_value, joined_trips_visitors_travel_time_value, joined_trips_passers_travel_time_value, joined_trips_nonAffectedAgents_travel_time_value, joined_trips_capturedAgents_travel_time_value, joined_trips_allAgents_travel_time_value)
travel_distance <- c(joined_trips_residents_travel_distance_value, joined_trips_workers_travel_distance_value, joined_trips_visitors_travel_distance_value, joined_trips_passers_travel_distance_value, joined_trips_nonAffectedAgents_travel_distance_value, joined_trips_capturedAgents_travel_distance_value, joined_trips_allAgents_travel_distance_value)

output_value <- data.frame(agent_group, number_agent_group, travel_time, travel_distance)
colnames(output_value) <- c("agent_type", "number_agent_group", "travel_time", "travel_distance")
outputDirectory = paste("events_Biao", runID, "MobilityAnalysis_perTrip.csv", sep = "/")
write.csv(output_value, outputDirectory, row.names = FALSE)